In [1]:
import os, sys
import cv2
sys.path.insert(0, os.path.abspath('../DECA/'))

In [2]:
folders = []

path = r"C:\Users\pojzi\Programovanie\bakalarka\impact_recognition\data"

reconstructed_folders_path = os.path.join(path, "neutral_images_3d")

input_files = [os.path.join(path, "neutral_images", f.split("\\")[-1]+ ".jpg") for f in os.listdir(reconstructed_folders_path) if os.path.isdir(os.path.join(reconstructed_folders_path, f))]


expressions_path = r"C:\Users\pojzi\Programovanie\bakalarka\impact_recognition\data\expressions\raw_data"
expressions_folders = [os.path.join(expressions_path, f) for f in os.listdir(expressions_path) if os.path.isdir(os.path.join(expressions_path, f))]
expressions_folders

expressions = [x.split('\\')[-1] for x in expressions_folders]

pattern = r"_[0-9]"

expression_image_map = {}

for expression, expression_folder in zip(expressions, expressions_folders):
    cropped_path = os.path.join(expression_folder, "cropped")
    # in cropped path find image ending with *m.jpg"
    for file in os.listdir(cropped_path):
        if file.endswith("m.jpg"):
            # new_file = re.sub(pattern, "", file)
            full_path = os.path.join(cropped_path, file)
            expression_image_map[expression] = full_path
            break

final_expressions_input_path = os.path.join("/".join(expressions_path.split('\\')[:-1]),  "training").replace("\\", "/")

for key, value in expression_image_map.items():
    filename = key + ".jpg"
    image = cv2.imread(value)
    cv2.imwrite(os.path.join(final_expressions_input_path, filename), image)

In [3]:
import sys
sys.path.append(r"C:\Users\pojzi\Programovanie\bakalarka\DECA")
from config import DECAConfig

args = DECAConfig()
args.rasterizer_type="pytorch3d"
args.saveObj = True
args.useTex = True
args.extractText = True
args.image_path = "TestSamples/upload/Gazdik2.png"

from decalib.deca import DECA
from decalib.datasets import datasets 
from decalib.utils import util
from decalib.utils.config import cfg as deca_cfg
import cv2
import torch

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cuda'

deca_cfg.model.use_tex = args.useTex
deca_cfg.rasterizer_type = args.rasterizer_type
deca_cfg.model.extract_tex = args.extractText

deca = DECA(config=deca_cfg, device=device)


c:\python37\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")
c:\python37\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\python37\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


creating the FLAME Decoder
trained model found. load c:\Users\pojzi\Programovanie\bakalarka\DECA\data\deca_model.tar


In [4]:
for param in deca.parameters():
    param.requires_grad = False  # Disable gradient calculation to prevent unnecessary memory allocations

In [5]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:1024'
import random

expdata_all = []

exp_list = ['anger', 'confusion', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']

expdata = datasets.TestData(final_expressions_input_path, iscrop=args.iscrop, face_detector=args.detector)
exp_images = [expdata[i]['image'].to(device)[None,...] for i in range(len(expdata))]


print("Loaded all expressions")
used = set()
start_batch = 7
print()
batch = len(input_files)
testdata = datasets.TestData(os.path.join(r"C:\Users\pojzi\Programovanie\bakalarka\impact_recognition\data", "neutral_images"), iscrop=args.iscrop, face_detector=args.detector)

cur_progress =  start_batch * len(exp_list)
max_progress = batch * len(exp_list)
for i in range(start_batch, len(input_files[:batch])):
    torch.cuda.empty_cache()
    
    sample = random.randint(0, len(testdata) - 1)
    while sample in used:
        sample = random.randint(0, len(testdata) - 1)
    
    used.add(sample)


    savefolder_root = os.path.join(r"C:\Users\pojzi\Programovanie\bakalarka\impact_recognition\data\output" , input_file.split("\\")[-1].split(".")[0])

    i = 0
    name = testdata[sample]['imagename']
    # savepath = '{}/{}.jpg'.format(savefolder, name)
    images = testdata[sample]['image'].to(device)[None,...]
    with torch.no_grad():
        id_codedict = deca.encode(images)
    id_opdict, id_visdict = deca.decode(id_codedict)
    id_visdict = {x:id_visdict[x] for x in ['inputs', 'shape_detail_images']}   

    # exp_list = ['8']
    print()
    print("Reconstructing expression ", name)
    for i, x in enumerate(exp_list):
        cur_progress += 1
        print(f"Progress: {cur_progress}/{max_progress}")

        savefolder = os.path.join(savefolder_root, x)
        # check if savefolder already exists
        if os.path.exists(savefolder):
            print("Skipping name ", name, " expression ", x, " as it already exists")
            continue
        os.makedirs(savefolder, exist_ok=True)

        # -- expression transfer
        # exp code from image
        exp_image = exp_images[i] #expdata[i]['image'].to(device)[None,...]

        with torch.no_grad():
            exp_codedict = deca.encode(exp_image)
        # transfer exp code
        id_codedict['pose'][:,3:] = exp_codedict['pose'][:,3:]
        id_codedict['exp'] = exp_codedict['exp']
        transfer_opdict, transfer_visdict = deca.decode(id_codedict)
        id_visdict['transferred_shape'] = transfer_visdict['shape_detail_images']
        cv2.imwrite(os.path.join(savefolder, name + '_animation.jpg'), deca.visualize(id_visdict))

        transfer_opdict['uv_texture_gt'] = id_opdict['uv_texture_gt']
        if args.saveDepth or args.saveKpt or args.saveObj or args.saveMat or args.saveImages:
            os.makedirs(os.path.join(savefolder, name, 'reconstruction'), exist_ok=True)
            os.makedirs(os.path.join(savefolder, name, 'animation'), exist_ok=True)
        
        # -- save results
        image_name = name
        for save_type in ['reconstruction', 'animation']:
            if type == "reconstruction": 
                continue
            
            if save_type == 'reconstruction':
                visdict = id_codedict; opdict = id_opdict
            else:
                visdict = transfer_visdict; opdict = transfer_opdict
            if args.saveDepth:
                depth_image = deca.render.render_depth(opdict['trans_verts']).repeat(1,3,1,1)
                visdict['depth_images'] = depth_image
                cv2.imwrite(os.path.join(savefolder, name, save_type, name + '_depth.jpg'), util.tensor2image(depth_image[0]))
            if args.saveKpt:
                np.savetxt(os.path.join(savefolder, name, save_type, name + '_kpt2d.txt'), opdict['landmarks2d'][0].cpu().numpy())
                np.savetxt(os.path.join(savefolder, name, save_type, name + '_kpt3d.txt'), opdict['landmarks3d'][0].cpu().numpy())
            if args.saveObj:
                deca.save_obj(os.path.join(savefolder, name, save_type, name + '.obj'), opdict)
            if args.saveMat:
                opdict = util.dict_tensor2npy(opdict)
                savemat(os.path.join(savefolder, name, save_type, name + '.mat'), opdict)
            if args.saveImages:
                for vis_name in ['inputs', 'rendered_images', 'albedo_images', 'shape_images', 'shape_detail_images']:
                    if vis_name not in visdict.keys():
                        continue
                    image  =util.tensor2image(visdict[vis_name][0])
                    cv2.imwrite(os.path.join(savefolder, name, save_type, name + '_' + vis_name +'.jpg'), util.tensor2image(visdict[vis_name][0]))

            print("Finished transferring expression ", x)
        del exp_image
        
        
    print(f'-- please check the results in {savefolder}')

Loaded all expressions



c:\Users\pojzi\.conda\envs\openpoints\lib\site-packages\torch\nn\functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)



Reconstructing expression  CFD-WM-244-003-N
Progress: 50/728
Skipping name  CFD-WM-244-003-N  expression  anger  as it already exists
Progress: 51/728
Skipping name  CFD-WM-244-003-N  expression  confusion  as it already exists
Progress: 52/728
Skipping name  CFD-WM-244-003-N  expression  disgust  as it already exists
Progress: 53/728
Skipping name  CFD-WM-244-003-N  expression  fear  as it already exists
Progress: 54/728
Skipping name  CFD-WM-244-003-N  expression  happiness  as it already exists
Progress: 55/728
Skipping name  CFD-WM-244-003-N  expression  sadness  as it already exists
Progress: 56/728
Skipping name  CFD-WM-244-003-N  expression  surprise  as it already exists
-- please check the results in C:\Users\pojzi\Programovanie\bakalarka\impact_recognition\data\output\CFD-AF-208-003-N\surprise

Reconstructing expression  CFD-IF-618-212-N
Progress: 57/728
Skipping name  CFD-IF-618-212-N  expression  anger  as it already exists
Progress: 58/728
Skipping name  CFD-IF-618-212-N 

In [6]:
data = expdata[0]

In [7]:
len(expdata)

7

In [8]:
# shuffle the data
import random
random.seed(0)
random.shuffle(testdata)
len(testdata)

TypeError: 'TestData' object does not support item assignment

In [ ]:
import torch
from pytorch3d.io import load_objs_as_meshes
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    TexturesVertex,
    Materials
)

from pytorch3d.renderer import PointLights, DirectionalLights, AmbientLights
from pytorch3d.renderer import BlendParams

device = args.device

# Assuming 'device' is defined (e.g., device = torch.device("cuda:0"))
# Load your mesh (make sure your .obj file's directory contains the .mtl and texture files)
# mesh = load_objs_as_meshes(["TestSamples/animation_results/Marek2_soldier/Marek2/animation/Marek2.obj"], device=device, load_textures=True)
verts, faces, aux = load_obj(f="TestSamples/animation_results/Marek2_soldier/Marek2/animation/Marek2.obj",
                            load_textures=True,
                            create_texture_atlas=True,
                            texture_wrap="repeat",
                            device=device)

raise Exception("done")


# Set up the renderer
R, T = look_at_view_transform(dist=0.7, elev=0, azim=0)  # Adjust `elev` and `azim` to change the camera angle
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)
lights = PointLights(device=device, location=[[0.0, 0.0, 3.0]])

blend_params = BlendParams(background_color=[0., 0., 0.])

raster_settings = RasterizationSettings(
    image_size=1024, 
    blur_radius=0.0, 
    faces_per_pixel=1,
)

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(
        cameras=cameras, 
        raster_settings=raster_settings,
    ),
    shader = SoftPhongShader(device=device, cameras=cameras, lights=lights, blend_params=blend_params)
    
)

materials = Materials(device=device, specular_color=[[1., 1., 1.]], shininess=10.)


# Render the image
image = renderer(mesh, materials=materials)

# Convert the rendered image for visualization
image = image.cpu().numpy()
plt.figure(figsize=(10, 10))
plt.imshow(image.squeeze())
plt.axis("off")

OutOfMemoryError: CUDA out of memory. Tried to allocate 7.31 GiB (GPU 0; 6.00 GiB total capacity; 38.17 MiB already allocated; 3.34 GiB free; 44.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
input_files[0]

'C:\\Users\\pojzi\\Programovanie\\bakalarka\\impact_recognition\\data\\neutral_images\\CFD-AF-201-060-N.jpg'

In [ ]:
torch.cuda.empty_cache()


In [ ]:
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    3854 MB |    4421 MB |  101934 MB |   98079 MB |\n|       from large pool |    3458 MB |    4025 MB |   95243 MB |   91784 MB |\n|       from small pool |     395 MB |     418 MB |    6690 MB |    6294 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |    3854 MB |    4421 MB |  101934 MB |   98079 MB |\n|       from large pool |    3458 MB |    4025 MB |

In [ ]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    3854 MB |    4421 MB |  101934 MB |   98079 MB |
|       from large pool |    3458 MB |    4025 MB |   95243 MB |   91784 MB |
|       from small pool |     395 MB |     418 MB |    6690 MB |    6294 MB |
|---------------------------------------------------------------------------|
| Active memory         |    3854 MB |    4421 MB |  101934 MB |   98079 MB |
|       from large pool |    3458 MB |    4025 MB |   95243 MB |

In [ ]:
cuda_config = torch.cuda.memory_reserved(0)

cuda_config['max_split_size_mb'] = 4685037568 * 2
torch.cuda.set_memory_reserved(0, cuda_config)


TypeError: 'int' object does not support item assignment

In [ ]:
cuda_config

4685037568

In [ ]:
x = "PYTORCH_CUDA_ALLOC_CONF"
os.environ[x]

KeyError: 'PYTORCH_CUDA_ALLOC_CONF'